In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/gdrive')
project_path = '/content/gdrive/MyDrive/master/3d_shapes_estimator'
sys.path.append(project_path)

In [ ]:
# reference: https://keras.io/examples/vision/depth_estimation/
class DataLoader(tf.keras.utils.Sequence):
  def __init__(self, data, batch_size=15, shuffle=False):
    """
    Initialization
    """
    self.shuffle = shuffle
    self.data = data
    self.x1_file = self.data[0]
    self.x2_file = self.data[1]
    self.y_file = self.data[2]
    self.x0_file = self.data[3]
    self.indices = list(range(self.x1_file.len()))
    self.len_indices = len(self.indices)
    self.batch_size = batch_size
    self.on_epoch_end()

  def __len__(self):
    if IS_SAMPLE == True:
      return int(np.ceil(self.len_indices / self.batch_size / SAMPLE_RATE))
    else:
      return int(np.ceil(self.len_indices / self.batch_size))

  def __getitem__(self, index):
    # modify batch size of last batch
    batch_size = self.batch_size
    if (index + 1) * self.batch_size > len(self.indices):
      batch_size = len(self.indices) - index * self.batch_size
    # Generate one batch of data
    # Generate indices of the batch
    index = self.indices[index * batch_size : (index + 1) * batch_size]
    # Find list of IDs
    x1, x2, y, x0 = self.load_batch(index)
    return x1, x2, y, x0


  def on_epoch_end(self):
    """
    Updates indexes after each epoch
    """
    self.index = np.arange(len(self.indices))
    if self.shuffle == True:
        np.random.shuffle(self.index)

  def load_batch(self, batch):
    """
    Load one batch of data.
    """
    x1 = self.x1_file[batch[0]:batch[-1]+1]
    x2 = self.x2_file[batch[0]:batch[-1]+1]
    y = (self.y_file[batch[0]:batch[-1]+1]).astype(np.float16)
    x0 = self.x0_file[batch[0]:batch[-1]+1]
    return x1, x2, y, x0

In [ ]:
import h5py

In [ ]:
hdf5_path = '/content/gdrive/MyDrive/master/Data/half_data_shuffled.hdf5'
train_dataset_file = (h5py.File(hdf5_path)['train_normal'], h5py.File(hdf5_path)['train_depth'], h5py.File(hdf5_path)['train_shape'], h5py.File(hdf5_path)['train_raw_image'])
test_dataset_file = (h5py.File(hdf5_path)['test_normal'], h5py.File(hdf5_path)['test_depth'], h5py.File(hdf5_path)['test_shape'], h5py.File(hdf5_path)['test_raw_image'])
validation_dataset_file = (h5py.File(hdf5_path)['validation_normal'], h5py.File(hdf5_path)['validation_depth'], h5py.File(hdf5_path)['validation_shape'], h5py.File(hdf5_path)['validation_raw_image'])

In [ ]:
metric_hist = []

# # load model
# loaded_generator = tf.saved_model.load(OUTPUT_PATH+'gen_weights03.ckpt')
# loaded_discriminator = tf.saved_model.load(OUTPUT_PATH+'disc_weights03.ckpt')
# # loaded_generator = tf.keras.models.load_model(OUTPUT_PATH+'gen_weights')
# # loaded_discriminator = tf.keras.models.load_model(OUTPUT_PATH+'disc_weights')

generator.load_weights(OUTPUT_PATH+'gen_weights%02d.ckpt' % curr_epoch)
discriminator.load_weights(OUTPUT_PATH+'disc_weights%02d.ckpt' % curr_epoch)
loaded_generator = generator
loaded_discriminator = discriminator

loaded_model = ShapeEstimator(generator=loaded_generator, discriminator=loaded_discriminator, gp_weight=GP_WEIGHT)
loaded_model.compile(gen_optimizer=gen_optimizer, disc_optimizer=disc_optimizer, metric=[tf.keras.metrics.RootMeanSquaredError(), earth_mover_loss])

# loaded_model = model

# test batches
count = 0
for step, batch in enumerate(test_loader):
  # save original image
  if step % 10 == 0:
    try:
      os.makedirs(OUTPUT_PATH+'predicted_shapes_test')
    except FileExistsError:
      pass
    with open(OUTPUT_PATH+'predicted_shapes_test/real_raw_image%02d.npz' % count, 'wb+') as f:
      np.save(f, batch[3])
    f.close()
    with open(OUTPUT_PATH+'predicted_shapes_test/real_normal_image%02d.npz' % count, 'wb+') as f:
      np.save(f, batch[0])
    f.close()
    with open(OUTPUT_PATH+'predicted_shapes_test/real_depth_image%02d.npz' % count, 'wb+') as f:
      np.save(f, batch[1])
    f.close()
    with open(OUTPUT_PATH+'predicted_shapes_test/predicted_normal_image%02d.npz' % count, 'wb+') as f:
      np.save(f, sketch_estimator.predict(batch[3]))
    f.close()
    with open(OUTPUT_PATH+'predicted_shapes_test/predicted_depth_image%02d.npz' % count, 'wb+') as f:
      np.save(f, sketch_estimator_depth.predict(batch[3]))
    f.close()

  # predict sketches
  # batch = list(batch)
  # batch[0] = sketch_estimator.predict(batch[0])
  # batch = tuple(batch)

  # concatenate sketches
  concatenated = np.concatenate((batch[0], batch[1]), axis=3)
  batch = (concatenated, batch[2])

  metric, shape_pred = loaded_model.test_step(batch)
  metric_hist.append(metric)

  # save sample predicted shapes and real shapes
  if step % 10 == 0:
    try:
      os.makedirs(OUTPUT_PATH+'predicted_shapes_test')
    except FileExistsError:
      pass
    with open(OUTPUT_PATH+'predicted_shapes_test/predicted_shape%02d.npz' % count, 'wb+') as f:
      np.save(f, shape_pred)
    f.close()
    with open(OUTPUT_PATH+'predicted_shapes_test/real_shape%02d.npz' % count, 'wb+') as f:
      np.save(f, batch[1])
    f.close()
    # with open(OUTPUT_PATH+'predicted_shapes_test/predicted_image%02d.npz' % count, 'wb+') as f:
    #   np.save(f, batch[0])
    # f.close()
    print('Shape saved at step %02d' % step)
    count += 1
avg_metric_MAE = sum(metric_hist[0]) / len(metric_hist[0])
avg_metric_EM = sum(metric_hist[1]) / len(metric_hist[1])
print('Average metric MAE: ' + str(avg_metric_MAE))
print('Average metric EM: ' + str(avg_metric_EM))